<a href="https://colab.research.google.com/github/VicDc/VIC_/blob/OPIT/VDC_A3_Smart_Home_AI_HF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install dependencies (IPython-friendly)
%pip install transformers torch -q
print("Dependencies installed.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.3 MB/s eta 0:00:00
Dependencies installed.


In [2]:
# -----------------------------------------------------------------------------
# AI Integration with Hugging Face
# -----------------------------------------------------------------------------
from transformers import AutoTokenizer, AutoModel
import torch

model_name = "Musixmatch/umberto-wikipedia-uncased-v1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def embed_command(command: str):
    """Generate a semantic embedding for the input command."""
    inputs = tokenizer(command, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    # Mean pooling over token embeddings
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

print("Hugging Face model loaded:", model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/309 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/801k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.02M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Hugging Face model loaded: Musixmatch/umberto-wikipedia-uncased-v1


In [3]:
class Light:
    VALID_STATES = ('on', 'off')

    def __init__(self):
        self.state = 'off'

    def set_state(self, state):
        if state in self.VALID_STATES:
            self.state = state
            print(f"Light turned {state}.")
        else:
            print(f"Invalid state for Light: {state}")

    def get_status(self):
        print(f"Light is currently {self.state}.")


class Fan:
    VALID_SPEEDS = ('low', 'medium', 'high')

    def __init__(self):
        self.speed = 'off'

    def set_state(self, state):
        if state in ('on', 'off'):
            if state == 'off':
                self.speed = 'off'
                print("Fan turned off.")
            else:
                self.speed = 'medium'
                print("Fan turned on at medium speed.")
        elif state in self.VALID_SPEEDS:
            self.speed = state
            print(f"Fan speed set to {state}.")
        else:
            print(f"Invalid state or speed for Fan: {state}")

    def get_status(self):
        print(f"Fan is set to {self.speed}.")


class Thermostat:
    def __init__(self):
        self.temperature = 20.0

    def set_temperature(self, temp):
        try:
            temp = float(temp)
            self.temperature = temp
            print(f"Thermostat set to {temp}°C.")
        except ValueError:
            print(f"Invalid temperature value: {temp}")

    def get_status(self):
        print(f"Thermostat is currently set at {self.temperature}°C.")

In [4]:
import re

def parse_command(command):
    command_lower = command.lower()

    # Use AI embedding for advanced processing (not used in this basic parser)
    # embedding = embed_command(command)

    if "light" in command_lower:
        device = 'light'
        if "status" in command_lower:
            return {'intent': 'get_status', 'device_type': device}
        if "turn on" in command_lower or "switch on" in command_lower:
            return {'intent': 'set_state', 'device_type': device, 'value': 'on'}
        if "turn off" in command_lower or "switch off" in command_lower:
            return {'intent': 'set_state', 'device_type': device, 'value': 'off'}

    if "fan" in command_lower:
        device = 'fan'
        if "status" in command_lower:
            return {'intent': 'get_status', 'device_type': device}
        if "turn on" in command_lower or "switch on" in command_lower:
            return {'intent': 'set_state', 'device_type': device, 'value': 'on'}
        if "turn off" in command_lower or "switch off" in command_lower:
            return {'intent': 'set_state', 'device_type': device, 'value': 'off'}
        for speed in Fan.VALID_SPEEDS:
            if speed in command_lower:
                return {'intent': 'set_state', 'device_type': device, 'value': speed}

    if "thermostat" in command_lower or "temperature" in command_lower:
        device = 'thermostat'
        if "status" in command_lower:
            return {'intent': 'get_status', 'device_type': device}
        match = re.search(r'(\d+)', command_lower)
        if match:
            return {'intent': 'set_temperature', 'device_type': device, 'value': match.group(1)}

    if "exit" in command_lower or "quit" in command_lower:
        return {'intent': 'exit'}

    return {'intent': 'unrecognized'}

In [5]:
def execute(parsed, devices):
    intent = parsed.get('intent')
    device_type = parsed.get('device_type')

    if intent == 'exit':
        print("Exiting Smart Home CLI.")
        return False

    if intent == 'get_status' and device_type:
        devices[device_type].get_status()
    elif intent == 'set_state' and device_type:
        devices[device_type].set_state(parsed.get('value'))
    elif intent == 'set_temperature' and device_type:
        devices[device_type].set_temperature(parsed.get('value'))
    else:
        print("Sorry, I didn't understand that command.")

    return True

In [ ]:
def run_smart_home_cli():
    """
    Starts and manages the Smart Home CLI.
    (Light, Fan, Thermostat classes + parser & executor are defined above.)
    """
    devices = {
        'light': Light(),
        'fan': Fan(),
        'thermostat': Thermostat()
    }
    print("Welcome to the Smart Home CLI. Type 'exit' to quit.")

    while True:
        command = input(">> ")
        parsed = parse_command(command)
        if not execute(parsed, devices):
            break


if __name__ == '__main__':
    run_smart_home_cli()

Welcome to the Smart Home CLI. Type 'exit' to quit.
Light is currently off.
Sorry, I didn't understand that command.
Sorry, I didn't understand that command.
Thermostat is currently set at 20.0°C.
Sorry, I didn't understand that command.
Light turned on.
Thermostat set to 26.0°C.
